# Shawn Trewartha
# ADSP 32021
# Assignment 3


## Questions and Answers -

What are the reported top 5 features?

> Based on a feature importance algorithm used it was determined that the top 5 important features are: Gender, Height, Weight, Schedule and Experience.

## FLAML

What are the top 3 models per validation score when using:  
 - all features  
 - only the top features (if you have to choose a number put in 3)

 > I chose to use FLAML as an autoML system, and I could not find a suitable way of determining a hierarchy of the models, rather than just the top performing model. So for this question I will only be providing the 1 top model:

>  - Using all features: **xgb_limitdepth** Test MSE: 20779.69559632951  
>  - Using top features: **xgb_limitdepth** Test MSE: 20779.69559632951  

What are the top 3 models per speed when using
  - all features
  - only the top features (if you have to choose a number put in 3)

> To determine the speed I chose to compare the speed of the following 3 models:lgbm, xgboost, and rf

> The Best models based in speed using all the features are:
>  - **lgbm** - 301.51 sec   
>  - **xgboost** - 302.99 sec  
>  - **rf** - 376.04 sec

> The Best models based in speed using the top features are:

> - **lgbm** - 301.51 sec   
> - **xgboost** - 302.58 sec  
> - **rf** - 398.89 sec

## H2O

What are the top 3 models per validation score when using:  
 - all features  
 - only the top features (if you have to choose a number put in 3)

 >  - Using all features:
    - **Stacked Ensemble Models** rmse = 329.075
    - **XgBoost rmse** = 329.184
    - **Gradient Boost Machines** rmse = 329.69


>  - Using top features:
    - **Stacked Ensemble Models** rmse = 331.761  
    - **XgBoost** rmse = 333.792  
    - **Random Forest** rmse = 331.922


What are the top 3 models per speed when using
  - all features
  - only the top features (if you have to choose a number put in 3)

> The Best models based in speed using all the features are:
> - **Random Forest** - 1.38 sec
> - **Gradient Boost Machines** - 1.40 sec
> - **XGBoost** - 5.58 sec

> The Best models based in speed using the top features are:

> - **Random Forest** - 0.94 sec
> - **Gradient Boost Machines** - 1.13 sec
> - **XGBoost** - 5.54 sec



How does the top models compare to your previously developed model (assignments 1 and 2) in terms of validation score and speed?

> I believe that the models - Especially the stacked ensemble models found using H2O performed much better than any other model that I created using this dataset in the previous assignments, both in accuracy scores and computational speed.

Is your platform AutoML no-code/low-code/full-code and why?

> I believe that both of the AutoML systems that I chose would be low-code examples. I still needed to close up the system using FLAML and H2O but I did not need to code out all of the possible algorithms that were attempted.


## Lazy Predict

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load data into pandas DF

import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/MyDrive/mlops/athletes.csv')

In [3]:
def data_clean(data):
    # Remove not relevant columns
    data = data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                               'train','background','experience','schedule','howlong', \
                               'deadlift','candj','snatch','backsq','experience',\
                               'background','schedule','howlong'])
    data = data.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',\
                              'filthy50','fgonebad','run400','run5k','pullups','train'])

    # Remove Outliers

    data = data[data['weight'] < 1500]
    data = data[data['gender'] != '--']
    data = data[data['age'] >= 18]
    data = data[(data['height'] < 96) & (data['height'] > 48)]

    data = data[(data['deadlift'] > 0) & (data['deadlift'] <= 1105)|((data['gender'] == 'Female') \
                 & (data['deadlift'] <= 636))]
    data = data[(data['candj'] > 0) & (data['candj'] <= 395)]
    data = data[(data['snatch'] > 0) & (data['snatch'] <= 496)]
    data = data[(data['backsq'] > 0) & (data['backsq'] <= 1069)]

    # Clean Survey Data

    decline_dict = {'Decline to answer|': np.nan}
    data = data.replace(decline_dict)
    data = data.dropna(subset=['background','experience','schedule','howlong','eat'])

    return data

In [4]:
# Install AutoKeras
!pip install lazypredict

In [5]:
data = data_clean(data)

data.head()

,region,gender,age,height,weight,candj,snatch,deadlift,backsq,eat,background,experience,schedule,howlong
21,Southern California,Male,30.0,71.0,200.0,235.0,175.0,385.0,315.0,I eat whatever is convenient|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 1x a week|I ty...,1-2 years|
22,Africa,Male,28.0,70.0,176.0,187.0,134.0,335.0,254.0,I eat 1-3 full cheat meals per week|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 1x a week|,2-4 years|
27,North East,Male,35.0,68.0,225.0,285.0,205.0,440.0,405.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I typically rest 4 or more days per month|,2-4 years|
50,North Central,Male,36.0,71.0,199.0,267.0,212.0,485.0,390.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|I p...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,1-2 years|
60,North East,Male,36.0,64.0,155.0,245.0,180.0,415.0,385.0,I eat strict Paleo|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 2x a week|I st...,4+ years|


In [6]:
data['Total Lift'] = data[['candj', 'snatch', 'deadlift', 'backsq']].sum(axis=1)
data = data.drop(columns=['candj', 'snatch', 'deadlift', 'backsq'])

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
import matplotlib.pyplot as plt
import numpy as np

In [8]:
X = data.drop(columns='Total Lift')
y = data['Total Lift']

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)

models, predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)

  0%|          | 0/42 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 290
[LightGBM] [Info] Number of data points in the train set: 24023, number of used features: 3
[LightGBM] [Info] Start training from score 1018.182991
                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
LGBMRegressor                                0.51       0.54   188.78   
GradientBoostingRegressor                    0.51       0.54   188.96   
HistGradientBoostingRegressor                0.51       0.53   189.14   
MLPRegressor                                 0.50       0.53   190.88   
XGBRegressor                                 0.50       0.52   191.28   
SVR                                          0.49       0.52   192.97   
NuSVR                                        0.48       0.51   193.97   


## Auto ML using FLAML

##### Install and Load all proper libraries and packags

In [10]:
!pip install -q optuna
!pip install -q flaml
!pip install -q --upgrade "flaml[autozero]" lightgbm openml pyspark pandas

In [11]:
from flaml import AutoML
import pandas as pd
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error

##### Import Data

In [12]:
data = pd.read_csv('/content/drive/MyDrive/mlops/athletes.csv')

data = data_clean(data)

data['Total Lift'] = data[['candj', 'snatch', 'deadlift', 'backsq']].sum(axis=1)
data = data.drop(columns=['candj', 'snatch', 'deadlift', 'backsq'])

##### Pre-Process Data for AutoML

In [13]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_cols = ['region','gender','eat','background','experience','schedule','howlong']
numeric_cols = ['age','height','weight']

data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

scaler = StandardScaler()
data_encoded[numeric_cols] = scaler.fit_transform(data_encoded[numeric_cols])

X = data_encoded.drop(columns=['Total Lift'])
y = data_encoded['Total Lift']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [14]:
data.head()

,region,gender,age,height,weight,eat,background,experience,schedule,howlong,Total Lift
21,Southern California,Male,30.00,71.00,200.00,I eat whatever is convenient|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 1x a week|I ty...,1-2 years|,1110.00
22,Africa,Male,28.00,70.00,176.00,I eat 1-3 full cheat meals per week|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 1x a week|,2-4 years|,910.00
27,North East,Male,35.00,68.00,225.00,I eat quality foods but don't measure the amount|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I typically rest 4 or more days per month|,2-4 years|,1335.00
50,North Central,Male,36.00,71.00,199.00,I eat quality foods but don't measure the amount|,I played youth or high school level sports|I p...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,1-2 years|,1354.00
60,North East,Male,36.00,64.00,155.00,I eat strict Paleo|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 2x a week|I st...,4+ years|,1225.00


##### Fit AutoML using FLAML and Determine Best Model

The Best model based on accuracy using all of the Features is:

**xgb_limitdepth**  
Test MSE: 20779.69559632951  
Test MAE: 107.17323225988692


In [17]:
automl = AutoML()

automl.fit(
    X_train, y_train,
    task="regression",
    metric="rmse",
    time_budget=600,
    n_concurrent_trials=1,
    verbose=1,
    use_spark=False,
    use_ray=False
)


y_pred = automl.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Test MSE:", mse)
print("Test MAE:", mae)


print("Best ML learner:", automl.best_estimator)
print("Best hyperparameters:", automl.best_config)

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


Test MSE: 20779.69559632951
Test MAE: 107.17323225988692
Best ML learner: xgb_limitdepth
Best hyperparameters: {'n_estimators': 1405, 'max_depth': 8, 'min_child_weight': np.float64(36.524019639762436), 'learning_rate': np.float64(0.012339184096746812), 'subsample': 1.0, 'colsample_bylevel': np.float64(0.7136300050303552), 'colsample_bytree': np.float64(0.568134320782359), 'reg_alpha': np.float64(0.0021023107511362825), 'reg_lambda': np.float64(5.712408409590408)}


The Best models based in time are:

**lgbm** - 301.51 sec   
**xgboost** - 302.99 sec  
**rf** - 376.04 sec

In [18]:
import time
from flaml import AutoML

# List of learners you want to test
learners = ["xgboost", "rf", "lgbm"]

results = []

for learner in learners:
    automl_tmp = AutoML()

    start = time.time()
    automl_tmp.fit(
        X_train, y_train,
        task="regression",
        time_budget=300,       # small budget per learner
        estimator_list=[learner],
        verbose=0
    )
    elapsed = time.time() - start

    # Best model's RMSE on training data
    y_pred_train = automl_tmp.predict(X_train)
    rmse = ((y_train - y_pred_train) ** 2).mean() ** 0.5

    results.append({
        "learner": learner,
        "train_time_sec": elapsed,
        "train_rmse": rmse
    })

df_speed = pd.DataFrame(results).sort_values(by="train_time_sec")
top_3_fastest = df_speed.head(3)

print(top_3_fastest)

   learner  train_time_sec  train_rmse
2     lgbm          301.51      130.68
0  xgboost          302.99      111.32
1       rf          376.04       95.65


##### Use only the Top 5 important features

The following code determines the top 5 important features as determind by the XGBoost model:

**Gender, Weight, Experiance, Height, and Schedule**

In [19]:
best_model = automl.model.estimator
importances = best_model.get_booster().get_score(importance_type='gain')

importance_df = pd.DataFrame({
    'feature': list(importances.keys()),
    'importance': list(importances.values())
})

importance_df = importance_df.sort_values(by='importance', ascending=False)

print(importance_df.head(6))

                                               feature  importance
19                                         gender_Male 22560286.00
2                                               weight  1275289.88
75   experience_I began CrossFit with a coach (e.g....  1045866.12
62   experience_I began CrossFit with a coach (e.g....   978145.88
1                                               height   920068.00
122                        howlong_Less than 6 months|   723744.69


Use AutoML (FLAML) using only th top 5 features

In [20]:
data2 = data.drop(columns=['region', 'age', 'eat', 'background', 'howlong'])

In [21]:
categorical_cols = ['gender','experience','schedule']
numeric_cols = ['height','weight']

data_encoded2 = pd.get_dummies(data2, columns=categorical_cols, drop_first=True)

scaler = StandardScaler()
data_encoded2[numeric_cols] = scaler.fit_transform(data_encoded2[numeric_cols])

X2 = data_encoded.drop(columns=['Total Lift'])
y2 = data_encoded['Total Lift']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=123)

The Best model based on accuracy using all of the Features is:

**xgb_limitdepth**  
Test MSE: 20779.69559632951  
Test MAE: 107.17323225988692

In [23]:
automl = AutoML()

automl.fit(
    X2_train, y2_train,
    task="regression",
    metric="rmse",
    time_budget=600,
    n_concurrent_trials=1,
    verbose=1,
    use_spark=False,
    use_ray=False
)


y2_pred = automl.predict(X2_test)


mse = mean_squared_error(y2_test, y2_pred)
mae = mean_absolute_error(y2_test, y2_pred)

print("Test MSE:", mse)
print("Test MAE:", mae)


print("Best ML learner:", automl.best_estimator)
print("Best hyperparameters:", automl.best_config)

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


Test MSE: 20779.69559632951
Test MAE: 107.17323225988692
Best ML learner: xgb_limitdepth
Best hyperparameters: {'n_estimators': 1405, 'max_depth': 8, 'min_child_weight': np.float64(36.524019639762436), 'learning_rate': np.float64(0.012339184096746812), 'subsample': 1.0, 'colsample_bylevel': np.float64(0.7136300050303552), 'colsample_bytree': np.float64(0.568134320782359), 'reg_alpha': np.float64(0.0021023107511362825), 'reg_lambda': np.float64(5.712408409590408)}


The Best models based in time are:

**lgbm** - 301.51 sec   
**xgboost** - 302.58 sec  
**rf** - 398.89 sec

In [24]:
import time
from flaml import AutoML

# List of learners you want to test
learners = ["xgboost", "rf", "lgbm"]

results = []

for learner in learners:
    automl_tmp = AutoML()

    start = time.time()
    automl_tmp.fit(
        X2_train, y2_train,
        task="regression",
        time_budget=300,       # small budget per learner
        estimator_list=[learner],
        verbose=0
    )
    elapsed = time.time() - start

    # Best model's RMSE on training data
    y2_pred_train = automl_tmp.predict(X_train)
    rmse = ((y2_train - y2_pred_train) ** 2).mean() ** 0.5

    results.append({
        "learner": learner,
        "train_time_sec": elapsed,
        "train_rmse": rmse
    })

df_speed = pd.DataFrame(results).sort_values(by="train_time_sec")
top_3_fastest = df_speed.head(3)

print(top_3_fastest)

   learner  train_time_sec  train_rmse
2     lgbm          301.50      133.82
0  xgboost          302.58      111.32
1       rf          398.89       95.65


## H2O AutoML

##### Install and load all of the needed Libraries and Packages

In [25]:
!pip install h2o
!apt-get update -qq
!apt-get install -y openjdk-11-jdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 MB 3.3 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libxcomposite1 libxt-dev libxtst6 libxxf86dga1
  openjdk-11-jdk-headless openjdk-11-jre openjdk-11-jre-headless
  session-migration x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  at-spi2-cor

In [26]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd

##### Load and pre-process the data

In [27]:
data = pd.read_csv('/content/drive/MyDrive/mlops/athletes.csv')

data = data_clean(data)

data['Total Lift'] = data[['candj', 'snatch', 'deadlift', 'backsq']].sum(axis=1)
data = data.drop(columns=['candj', 'snatch', 'deadlift', 'backsq'])

##### H20 Modeling and Fitting

In [28]:
h2o.init(max_mem_size="6G")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.16" 2025-07-15; OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1); OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.12/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpv2a01ab2
  JVM stdout: /tmp/tmpv2a01ab2/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpv2a01ab2/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_unknownUser_3d1045
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [29]:
categorical_cols = ['region','gender','eat','background','experience','schedule','howlong']
numeric_cols = ['age','height','weight']

X = data.drop(columns=['Total Lift'])
y = data['Total Lift']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [30]:
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [31]:
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train.reset_index(drop=True)], axis=1))
test_h2o = h2o.H2OFrame(X_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [32]:
# Convert categorical columns to factors
for col in categorical_cols:
    train_h2o[col] = train_h2o[col].asfactor()
    test_h2o[col] = test_h2o[col].asfactor()

# Ensure target is numeric
y_col = 'Total Lift'
train_h2o[y_col] = train_h2o[y_col].asnumeric()


# Feature columns
X_cols = [col for col in train_h2o.columns if col != y_col]

In [33]:
aml = H2OAutoML(
    max_models=20,        # max models to train
    max_runtime_secs=600, # same time budget as FLAML
    seed=42,
    sort_metric="RMSE",
    exclude_algos=["DeepLearning"]  # optional to avoid failures on small datasets
)

aml.train(x=X_cols, y=y_col, training_frame=train_h2o)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),20/20
# GBM base models (used / total),8/8
# XGBoost base models (used / total),9/9
# DRF base models (used / total),2/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


##### Determine the top models based on accuracy using all features

The best models based on accuracy are:

**Stacked Ensemble Models** rmse = 329.075  
**XgBoost** rmse = 329.184  
**Gradient Boost Machines** rmse = 329.697

In [34]:
y_pred = aml.leader.predict(test_h2o).as_data_frame().values.flatten()

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Test MSE:", mse)
print("Test MAE:", mae)

print("Top 5 H2O AutoML models:")
print(aml.leaderboard.head(5))

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Test MSE: 131156.41401427655
Test MAE: 293.4351639274333
Top 5 H2O AutoML models:
model_id                                                    rmse     mse      mae     rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20251113_195818     329.075  108290  239.189  0.972362                    108290
StackedEnsemble_BestOfFamily_1_AutoML_1_20251113_195818  329.077  108292  239.376  0.972406                    108292
XGBoost_grid_1_AutoML_1_20251113_195818_model_4          329.184  108362  239.246  0.971325                    108362
XGBoost_3_AutoML_1_20251113_195818                       329.621  108650  239.15   0.971328                    108650
GBM_5_AutoML_1_20251113_195818                           329.697  108700  240.239  0.973417                    108700
[5 rows x 6 columns]



/usr/local/lib/python3.12/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


##### Determine the top models based on speed using all features

The top models based on speed using all the features are:  

**Random Forest** - 1.38 sec  
**Gradient Boost Machines** - 1.40 sec  
**XGBoost** - 5.58 sec

In [35]:
import time
import pandas as pd
from h2o.automl import H2OAutoML

algos = ["GBM", "XGBoost", "DRF"]

results = []

for algo in algos:
    aml = H2OAutoML(
        max_models=1,
        seed=42,
        exclude_algos=[a for a in algos if a != algo]
    )

    start = time.time()
    aml.train(x=X_cols, y=y_col, training_frame=train_h2o)
    elapsed = time.time() - start

    results.append({
        "model_type": algo,
        "train_time_sec": elapsed,
        "rmse": aml.leader.rmse()
    })


df_speed = pd.DataFrame(results).sort_values(by="train_time_sec")
top_3_by_speed = df_speed.head(3)

print("Top 3 models by speed:")
print(top_3_by_speed)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Top 3 models by speed:
  model_type  train_time_sec   rmse
2        DRF            1.38 330.26
0        GBM            1.40 330.26
1    XGBoost            5.58 315.33


##### H20 Using the top 5 features

In [45]:
data = pd.read_csv('/content/drive/MyDrive/mlops/athletes.csv')

data = data_clean(data)

data['Total Lift'] = data[['candj', 'snatch', 'deadlift', 'backsq']].sum(axis=1)
data = data.drop(columns=['candj', 'snatch', 'deadlift', 'backsq','region', 'age', 'eat', 'background', 'howlong'])

In [38]:
h2o.init(max_mem_size="6G")

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,6 mins 57 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_unknownUser_3d1045
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.900 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [48]:
categorical_cols = ['gender','experience','schedule']
numeric_cols = ['height','weight']

X = data.drop(columns=['Total Lift'])
y = data['Total Lift']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [49]:
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [50]:
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train.reset_index(drop=True)], axis=1))
test_h2o = h2o.H2OFrame(X_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [51]:
# Convert categorical columns to factors
for col in categorical_cols:
    train_h2o[col] = train_h2o[col].asfactor()
    test_h2o[col] = test_h2o[col].asfactor()

# Ensure target is numeric
y_col = 'Total Lift'
train_h2o[y_col] = train_h2o[y_col].asnumeric()


# Feature columns
X_cols = [col for col in train_h2o.columns if col != y_col]

In [52]:
aml = H2OAutoML(
    max_models=20,        # max models to train
    max_runtime_secs=600, # same time budget as FLAML
    seed=42,
    sort_metric="RMSE",
    exclude_algos=["DeepLearning"]  # optional to avoid failures on small datasets
)

aml.train(x=X_cols, y=y_col, training_frame=train_h2o)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),17/20
# GBM base models (used / total),6/8
# XGBoost base models (used / total),8/9
# DRF base models (used / total),2/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


##### Determine the top models based on accuracy using top features

**Stacked Ensemble Models** rmse = 331.761  
**XgBoost** rmse = 333.792  
**Random Forest** rmse = 331.922

In [53]:
y_pred = aml.leader.predict(test_h2o).as_data_frame().values.flatten()

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Test MSE:", mse)
print("Test MAE:", mae)

print("Top 5 H2O AutoML models:")
print(aml.leaderboard.head(5))

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Test MSE: 126385.49241394234
Test MAE: 285.4795159169184
Top 5 H2O AutoML models:
model_id                                                    rmse     mse      mae     rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_5_20251113_201117     331.761  110065  243.123  0.975122                    110065
StackedEnsemble_BestOfFamily_1_AutoML_5_20251113_201117  331.767  110069  243.467  0.975264                    110069
XGBoost_grid_1_AutoML_5_20251113_201117_model_4          331.792  110086  242.146  0.973261                    110086
DRF_1_AutoML_5_20251113_201117                           331.922  110173  241.025  0.973458                    110173
XGBoost_3_AutoML_5_20251113_201117                       332.13   110310  241.908  0.97363                     110310
[5 rows x 6 columns]



/usr/local/lib/python3.12/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


##### Determine the top models based on speed using all features

The top models based on speed using all the features are:  

**Random Forest** - 0.94 sec  
**Gradient Boost Machines** - 1.13 sec  
**XGBoost** - 5.54 sec

In [54]:
import time
import pandas as pd
from h2o.automl import H2OAutoML

algos = ["GBM", "XGBoost", "DRF"]

results = []

for algo in algos:
    aml = H2OAutoML(
        max_models=1,
        seed=42,
        exclude_algos=[a for a in algos if a != algo]
    )

    start = time.time()
    aml.train(x=X_cols, y=y_col, training_frame=train_h2o)
    elapsed = time.time() - start

    results.append({
        "model_type": algo,
        "train_time_sec": elapsed,
        "rmse": aml.leader.rmse()
    })


df_speed = pd.DataFrame(results).sort_values(by="train_time_sec")
top_3_by_speed = df_speed.head(3)

print("Top 3 models by speed:")
print(top_3_by_speed)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Top 3 models by speed:
  model_type  train_time_sec   rmse
2        DRF            0.94 332.09
0        GBM            1.13 332.09
1    XGBoost            5.54 324.94
